## 실행방법
1. colab의 런타임을 GPU로 바꾸어주세요.
2. 차례대로 코드를 실행합니다.
3. 예시 전까지의 코드를 실행한 후, 
4. 가장 마지막 부분에 실행코드에 원하는 context와 질문을 입력하고 엔터를 눌러주세요.
###주의사항
Optimizer을 통해 모델을 학습시키는 시간이 조금 오래걸릴 수 있습니다.   


# KcBERT QA Model

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6057250334689329171
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14415560704
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1581825597300047366
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import tqdm.notebook
import math

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained('beomi/kcbert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base')

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
qa = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

In [ ]:
context = '정답을 찾을 문맥을 입력해주세요'

In [ ]:
question = '질문을 입력해주세요'

In [ ]:
qa(question=question, context=context)

{'score': 0.034865111112594604, 'start': 7, 'end': 17, 'answer': '문맥을 입력해주세요'}

In [ ]:
qa(question='질문을 입력해주세요', context=context)

{'score': 0.034865111112594604, 'start': 7, 'end': 17, 'answer': '문맥을 입력해주세요'}

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, QuestionAnsweringPipeline
import tensorflow as tf
import urllib.request
import json

##KorQuAD 데이터셋 불러오기

In [ ]:
urllib.request.urlretrieve('https://korquad.github.io/dataset/KorQuAD_v1.0_train.json', 'korquad.json')

('korquad.json', <http.client.HTTPMessage at 0x7fb462387d30>)

In [ ]:
korquad = json.load(open('korquad.json', encoding='utf8'))

In [ ]:
korquad['data'][0]

{'paragraphs': [{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
     'id': '6566495-0-0',
     'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
    {'answers': [{'text': '1악장', 'answer_start': 421}],
     'id': '6566495-0-1',
     'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
    {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
     'id': '6566495-0-2',
     'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
    {'answers': [{'text': '파우스트', 'answer_start': 15}],
     'id': '6566518-0-0',
     'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
    {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
     'id': '6566518-0-1',
     'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
    {'answers': [{'text': '1839', 'answer_start': 0}],
     'id': '5917067-0-0',
     'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
    {'answers': [{'text': '파리', 'answer_start': 410}],
     'id': '5917067-0-1',
     'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
    {'answers': [{'text': '드레스덴', 'answer_sta

In [ ]:
para = korquad['data'][0]['paragraphs'][0]
para

{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
   'id': '6566495-0-0',
   'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
  {'answers': [{'text': '1악장', 'answer_start': 421}],
   'id': '6566495-0-1',
   'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
  {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
   'id': '6566495-0-2',
   'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
  {'answers': [{'text': '파우스트', 'answer_start': 15}],
   'id': '6566518-0-0',
   'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
  {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
   'id': '6566518-0-1',
   'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
  {'answers': [{'text': '1839', 'answer_start': 0}],
   'id': '5917067-0-0',
   'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
  {'answers': [{'text': '파리', 'answer_start': 410}],
   'id': '5917067-0-1',
   'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
  {'answers': [{'text': '드레스덴', 'answer_start': 534}],
   'id': '5917067-0-2',
   'question': '바그너의 1

In [ ]:
# 데이터셋의 context 불러오기
para['context']

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [ ]:
# 데이터셋의 질문과 답
qas = para['qas'][0]
qas

{'answers': [{'text': '교향곡', 'answer_start': 54}],
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'}

In [ ]:
inputs = tokenizer(para['context'], 
                   qas['question'], 
                   truncation=True, 
                   max_length=tokenizer.model_max_length, 
                   return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(1, 300), dtype=int32, numpy=
array([[    2,  8601,  4633, 29697,  1480,  4313,  4538,  4008,   336,
         4065,  4042,  3231, 23243,  4104,  4027,  8793, 13985,   391,
         9132,  4113, 10966, 11728, 12023, 14657,  4091,  8598, 16639,
          341,  4573,  4771,  4027,  2139,  8478, 14416,   214,  8202,
           17,  2451, 13007,  1480,  4313,  4538,  4008,  8601,  4633,
        22903,  4113,  1676,   868,  4913,  7965,  1789,  4203,  4110,
        15031,   786,   250,  4057, 10878,  4007,  2593,  4094,  4128,
        10289,  4113, 10958,  4062,  9511,  1355,  4600,  4103,  4775,
         5602, 10770,  4180, 26732,  3231, 23243,  4104,  4042,  2015,
         4012,  4113,  9198,  8763,  8129,    17, 10384, 23008,  7971,
         2170,  4408,  4011,  4147,  4042, 17015,  4091, 23008, 21056,
         4165,   323,  4175,  4158, 11413,  2273,  4043,  7966,  1543,
         4775,  4170,  4042,   341,  4573,  4771,    28,  4566,  4027,
        10599, 

In [ ]:
q = qas['answers'][0]

start_char = q['answer_start']
end_char = start_char + len(q['text']) - 1

start = inputs.char_to_token(0, start_char)
end = inputs.char_to_token(0, end_char)

In [ ]:
# 글자의 위치print(start_char)
print(end_char)

56


In [ ]:
print(start)
print(end)

27
29


In [ ]:
inputs['input_ids']

<tf.Tensor: shape=(1, 300), dtype=int32, numpy=
array([[    2,  8601,  4633, 29697,  1480,  4313,  4538,  4008,   336,
         4065,  4042,  3231, 23243,  4104,  4027,  8793, 13985,   391,
         9132,  4113, 10966, 11728, 12023, 14657,  4091,  8598, 16639,
          341,  4573,  4771,  4027,  2139,  8478, 14416,   214,  8202,
           17,  2451, 13007,  1480,  4313,  4538,  4008,  8601,  4633,
        22903,  4113,  1676,   868,  4913,  7965,  1789,  4203,  4110,
        15031,   786,   250,  4057, 10878,  4007,  2593,  4094,  4128,
        10289,  4113, 10958,  4062,  9511,  1355,  4600,  4103,  4775,
         5602, 10770,  4180, 26732,  3231, 23243,  4104,  4042,  2015,
         4012,  4113,  9198,  8763,  8129,    17, 10384, 23008,  7971,
         2170,  4408,  4011,  4147,  4042, 17015,  4091, 23008, 21056,
         4165,   323,  4175,  4158, 11413,  2273,  4043,  7966,  1543,
         4775,  4170,  4042,   341,  4573,  4771,    28,  4566,  4027,
        10599, 18907,   208, 

In [ ]:
tokenizer.decode(inputs['input_ids'].numpy()[0, start:end+1])

'교향곡'

In [ ]:
inputs['start_positions'] = [start]
inputs['end_positions'] = [end]

In [ ]:
def int_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def make_inputs(context, qas):
    inputs = tokenizer(
        context, 
        qas['question'], 
        truncation=True, 
        max_length=tokenizer.model_max_length)
    q = qas['answers'][0]
    start_char = q['answer_start']
    end_char = start_char + len(q['text']) - 1
    start = inputs.char_to_token(0, start_char)
    end = inputs.char_to_token(0, end_char)
    inputs['start_positions'] = [start]
    inputs['end_positions'] = [end]
    return inputs

In [ ]:
%%time
n = 0
filename = 'korquad.tfrecord'
with tf.io.TFRecordWriter(filename) as writer:
    for item in korquad['data']:  
        for para in item['paragraphs']: 
            context = para['context']
            for qas in para['qas']:
                inputs = make_inputs(context, qas)
                if inputs['start_positions'][0] and inputs['end_positions'][0]:
                    feature = {k: int_feature(v) for k, v in inputs.items()}
                    example = tf.train.Example(features=tf.train.Features(feature=feature))
                    s = example.SerializeToString()
                    writer.write(s)
                    n += 1

CPU times: user 46.9 s, sys: 1.01 s, total: 47.9 s
Wall time: 50.2 s


In [ ]:
int_seq = tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True)
int_value = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
feature_description = {
    'input_ids': int_seq,
    'token_type_ids': int_seq,
    'attention_mask': int_seq,
    'start_positions': int_value,
    'end_positions': int_value
}

In [ ]:
def preproc(example):
    example = tf.io.parse_single_example(example, feature_description)
    example = {k : tf.cast(v, tf.int32) for k, v in example.items()}
    return example

In [ ]:
dataset = tf.data.TFRecordDataset(['korquad.tfrecord']).map(preproc).padded_batch(8)
batch = next(iter(dataset))
result = model(batch)

In [ ]:
#adam optimizer
%%time
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)

n = 10
for batch in tqdm.notebook.tqdm(dataset, total=math.ceil(n / 32)):
    with tf.GradientTape() as tape:
        result = model(batch)
        loss = tf.reduce_mean(result['loss'])
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables)) # loss가 감소하는 방향으로 파라미터 업데이트

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 42min 12s, sys: 1min 32s, total: 43min 45s
Wall time: 1h 7min 11s


In [ ]:
def qa(question, context):
    inputs = tokenizer(context, question, add_special_tokens=True, return_tensors="tf")
    outputs = model(inputs)
    start = tf.argmax(outputs.start_logits, axis=1).numpy()[0]
    end = tf.argmax(outputs.end_logits, axis=1).numpy()[0]
    return tokenizer.decode(inputs['input_ids'][0, start:end+1])

#예시


In [ ]:
context="""피고인과 피해자 B(남, 40세)는 모두 서울 ○○구 ○○로 ○○에 있는 ○○스포츠센터의 수영 강습생이다.
피고인은 2020. 10. 29. 09:30경 위 ○○스포츠센터 수영장에서 수영 강습을 받던 중, 피고인의 옆에서 수영 강습을 받기 위해
 물에 엎드린 자세로 떠 있던 피해자의 엉덩이를 갑자기 손으로 움켜쥐었다. 이로써 피고인은 피해자를 강제로 추행하였다."""

In [ ]:
context="""피고인은 2020년 
12년 15일경 집에서 피해자D(여,22세)에게 
다가가 피해자의 엉덩이를 손으로 만져 
피해자를 강제추행하였다."""

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2020년 12년 15일'

In [ ]:
qa('어떤 범죄를 저질렀는지?', context)

'피해자를 강제추행'

In [ ]:
qa('범행이 어디서 발생했는지?',context)

'집에서'

In [ ]:
context="피고인은 2021. 3. 31. 17:35경 광주 동구 B, 1층에 있는 C은행 후문 옆 흡연장에서 담배를 피우고 있는 피해자 D(여, 22세)에게 다가가, 갑자기 피해자의 엉덩이를 오른손으로 치고 가 피해자를 강제추행하였다."

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2021. 3. 31. 17 : 35경 광주 동구 B, 1층에 있는 C은행 후문 옆 흡연장에서 담배를 피우고 있는 피해자 D ( 여, 22세 ) 에게 다가가, 갑자기 피해자의 엉덩이를 오른손으로 치고 가 피해자를 강제추행'

In [ ]:
qa('어떤 범죄를 저질렀는지?', context)

'강제추행'

In [ ]:
qa('어디에서 사건이 발생했는지?',context)

'광주'

In [ ]:
context="피고인은 2020. 1. 18. 22:00경 포항시 북구 B 소재 ‘C 노래연습장' 내에서 선배인 D과 D의 여자친구인 피해자 E(가명, 여, 67세)와 함께 노래를 부르며 놀던 중 D이 뒤돌아서서 노래를 선곡하는 틈을 이용하여 갑자기 양팔로 피해자를 끌어안은 후 한 손으로 피해자의 왼쪽 가슴을 만졌다. 이로써 피고인은 피해자를 강제로 추행하였다."

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2020'

In [ ]:
qa('어떤 범죄를 저질렀는지?', context)

'양팔로 피해자를'

In [ ]:
qa('어디에서 사건이 발생했는지?',context)

'양팔'

In [ ]:
qa('어떤 행위를 하였는지?',context)

'피해자를 강제로 추행'

In [ ]:
context="그는 그녀를 2020년 12월 강제추행하였다."

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2020년 12월'

In [ ]:
qa('어떤 행위를 하였는지?',context)

'강제추행'

In [ ]:
qa('어디에서 사건이 발생했는지?',context)

'강제추행'

In [ ]:
context="그는 그녀를 2020년 집에서 강제로 추행하였다."

In [ ]:
qa('어디에서 사건이 발생했는지?',context)

'2020년 집에서'

In [ ]:
qa('어떤 행위를 하였는지?',context)

'강제로'

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2020년 집에서'

In [ ]:
context="피고인은 피해자를 2020년12월15일경 집에서 강제추행하였다."

In [ ]:
qa('어디에서 사건이 발생했는지?',context)

'2020년12월15일경 집에서'

In [ ]:
qa('어떤 행위를 하였는지?',context)

'강제추행'

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2020년12월15일경'

In [ ]:
context="피고인과 피해자 B(여, 35세)은 같은 직장에 근무하는 동료 사이이다. 피고인은 2019. 10. 26. 23:00경 대구 수성구 C 앞길에서 동료들과 회식을 마치고 귀가하던 중 '나랑 같이 자자.'라고 말하며 갑자기 피해자의 왼쪽 가슴을 움켜잡아 피해자를 강제로 추행하였다."

In [ ]:
qa('어디에서 사건이 발생했는지?',context)

'대구 수성구 C 앞길에서 동료들과 회식'

In [ ]:
qa('어떤 행위를 하였는지?',context)

'회식'

In [ ]:
qa('범행이 언제 발생했는지?', context)

'2019. 10. 26. 23'

In [ ]:
qa('피고인과 피해자는 어떤 관계인가?',context)

"B ( 여, 35세 ) 은 같은 직장에 근무하는 동료 사이이다. 피고인은 2019. 10. 26. 23 : 00경 대구 수성구 C 앞길에서 동료들과 회식을 마치고 귀가하던 중'나랑 같이 자자.'라고 말하며 갑자기 피해자의 왼쪽 가슴"

In [ ]:
context = input('검색할 context를 입력해주세요:  ')
question=input('검색할 질문을 입력해주세요:  ')
print("정답")
qa(question, context)

검색할 context를 입력해주세요:  피고인은 2020년 12월 15일경 집에서 피해자D(여,22세)에게 다가가 피해자의 엉덩이를 손으로 만져 피해자를 강제추행하였다
검색할 질문을 입력해주세요:  범행이 언제 발생했는지?
정답


'2020년 12월 15일'

#검색을 위한 코드입니다. 실행한후, context와 Question을 차례대로 입력한 후 엔터를 눌러주세요.

In [ ]:
context = input('검색할 context를 입력해주세요:  ')
question=input('검색할 질문을 입력해주세요:  ')
print("정답")
qa(question, context)

검색할 context를 입력해주세요:  피고인은 2020년 12월 15일경 집에서 피해자D(여,22세)에게 다가가 피해자의 엉덩이를 손으로 만져 피해자를 강제추행하였다
검색할 질문을 입력해주세요:  범행이 언제 발생했나요?
정답


'2020년 12월 15일'